Загалом, структури даних діляться на дві великі категорії:

* лінійні (списки, множини, стеки, черги тощо)
* нелінійні (дерева, графи)

Вбудовані структури даних у Python є лінійними. Але, окрім них, існують багато інших лінійних структур даних.

Цього заняття ми розглянемо наступні:

* stack
* queue
* deque

## Stack

Стек це структура даних, в котрій доступ до елементів організований за принципом LIFO (last in, first out).

[Прекрасний приклад стеку з реального життя](https://upload.wikimedia.org/wikipedia/commons/1/19/Tallrik_-_Ystad-2018.jpg)

In [2]:
class CustomStack:
    def __init__(self):
        self._stack = list()
        
    def push(self, element):
        self._stack.append(element)
        
    def pop(self):
        return self._stack.pop()
    
    def empty(self):
        return len(self._stack) == 0
    
    def top(self):
        return self._stack[-1]


In [10]:
s = CustomStack()
s.push(1)
s.push(2)
s.push(3)
s.push(4)
s.push(5)

In [11]:
s.pop()

5

In [12]:
s.pop()

4

Загалом, ми можемо працювати зі звичайним списком як зі стеком, завдяки методам pop() і append().

Стек дуже активно використовується в програмуванні. Базові типи даних організовані в пам'яті як стек (а не базові організовані у heap, [прекрасне пояснення різниці](https://www.youtube.com/watch?v=IX3fDYz0WyM). Виклики функцій (в тому числі рекурсивні) організовані як стек. Саме тому, кожна рекурсія має прагнути до базового випадку. Він обчислюється першим, далі - випадок за ним і так до кінця. Тобто, саме __обчислення__ організовано за принципом LIFO.
Більш того, процес виконання програми частково організований як стек. Вкладені блоки коду обчислюються першими, далі - ті, що над ними і так до кінця.

Прекрасний приклад використання стеку - калькулятор на двох стеках від Едсгера Дейкстри. Наведена версія має певні обмеження (пов'язані з дужками), але демонструє загальну ідею.

In [5]:
expression = "(10*2)-(3+2)*12"

In [6]:
operators = "+-*/"
operators_to_funcs = {
    "+": lambda a, b: a + b,
    "-": lambda a, b: a - b,
    "*": lambda a, b: a*b,
    "/": lambda a, b: a/b
}

operators_stack = CustomStack()
operands_stack = CustomStack()

def compute_and_push_to_stack(operands_stack: CustomStack, operators_stack: CustomStack):
    operator = operators_stack.pop()
    operator_func = operators_to_funcs[operator]
    right_operand = operands_stack.pop()
    left_operand = operands_stack.pop()
    result = operator_func(left_operand, right_operand)
    operands_stack.push(result)


operand = ""

for char_index, char in enumerate(expression):
    if char.isnumeric():
        operand += char
    if char in operators:
        if operand != "":
            operands_stack.push(int(operand))
            operand = ""
        operators_stack.push(char)
    if char == ")" or char_index == len(expression) - 1:
        if operand != "":
            operands_stack.push(int(operand))
            operand = ""
        compute_and_push_to_stack(operands_stack, operators_stack)

while not operators_stack.empty():
    compute_and_push_to_stack(operands_stack, operators_stack)

result = operands_stack.pop()
print(result)

-40


## Queue

Черга це структура даних, в котрій доступ до елементів організований за принципом FIFO (first in, first out). Найочевиднішим прикладом буде черга в магазині.

In [7]:
class CustomQueue:
    def __init__(self):
        self._queue = list()
        
    def put(self, element):
        self._queue.append(element)
        
    def get(self):
        return self._queue.pop(0)
    
    def is_empty(self):
        return len(self._queue) == 0


In [13]:
q = CustomQueue()
q.put(1)
q.put(2)
q.put(3)
q.put(4)
q.put(5)

In [14]:
q.get()

1

In [16]:
q.get()

3

В Python існує вбудована імплементація черги з модуля queue. Бажано використовувати саме її, особливо для роботи з конкурентними програмами. Одна з причин в тому, що асимптотична складність операції pop(0) складає O(n).

In [17]:
from queue import Queue

q2 = Queue()

Черга використовується в основному для поступового виконання різних завдань. Черги можуть з'являтись не тільки в рамках програми, а в рамках взаємодії декількох серверів (так звані менеджери черг, наприклад, RabbitMQ).

З цього сценарію використання росте й архітектурний патерн під назвою Producer-Consumer. Він використовується для послідовної обробки невідомої кількості задач необмеженою кількістю виконавців. Він складається з трьох компонентів:

* Queue - черга, куди складаються задачі, котрі ще не були оброблені
* Producer - той, хто кладе задачі в Queue
* Consumer - той, хто вичитує задачі з Queue

In [18]:
list_of_statements_1 = ["This is a tasty burger", 
                        "I dare you, I double dare you",
                        "Say 'what' one more time!"]*10

list_of_statements_2 = ["I just wanna feel", 
                        "Real love", 
                        "Cause i got too much life", 
                        "Running through mah veins"]*10


In [19]:
task_queue = Queue()

In [20]:
from time import sleep
from random import randint
from typing import List


class Producer:
    """Class which reads some list and pushes messages into queue with some timeout"""
    
    def __init__(self, sequence: List[str], queue: Queue):
        self._sequence = sequence
        self._queue = queue
        
    def _push(self, message):
        self._queue.put(message)
        
    def process_sequence(self):
        for message in self._sequence:
            self._push(message)
            sleep(0.1)
            
            
class Consumer:
    """Class which reads messages from queue and prints it with a consumer ID"""
    
    def __init__(self, consumer_id: int, queue: Queue):
        self._id = consumer_id
        self._queue = queue
        
    def consume(self):
        retries = 0
        while True:
            if not self._queue.empty():
                message = self._queue.get()
                print(f"Received message '{message}' by consumer {self._id}")
                sleep(0.3)
            else:
                sleep(1)
                retries += 1
            if retries >= 10:
                print(f"Queue seems empty and producers seems stopped. Consumer {self._id} stopped")
                break

In [21]:
prod_1 = Producer(list_of_statements_1, task_queue)
prod_2 = Producer(list_of_statements_2, task_queue)

cons_1 = Consumer(1, task_queue) 
cons_2 = Consumer(2, task_queue)
cons_3 = Consumer(3, task_queue)

In [22]:
from threading import Thread

t1 = Thread(target = prod_1.process_sequence)
t2 = Thread(target = prod_2.process_sequence)

t3 = Thread(target = cons_1.consume)
t4 = Thread(target = cons_2.consume)
t5 = Thread(target = cons_3.consume)

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()

t1.join()
t2.join()
t3.join()
t4.join()
t5.join()

Received message 'This is a tasty burger' by consumer 1
Received message 'I just wanna feel' by consumer 2
Received message 'I dare you, I double dare you' by consumer 1
Received message 'Real love' by consumer 2
Received message 'Say 'what' one more time!' by consumer 1
Received message 'Cause i got too much life' by consumer 2
Received message 'This is a tasty burger' by consumer 1Received message 'Running through mah veins' by consumer 2

Received message 'I dare you, I double dare you' by consumer 3
Received message 'I just wanna feel' by consumer 2
Received message 'Say 'what' one more time!' by consumer 1
Received message 'Real love' by consumer 3
Received message 'This is a tasty burger' by consumer 2
Received message 'Cause i got too much life' by consumer 1
Received message 'I dare you, I double dare you' by consumer 3
Received message 'Running through mah veins' by consumer 2
Received message 'Say 'what' one more time!' by consumer 1
Received message 'I just wanna feel' by co

## Deque

Дека, або двохстороння черга (Double Ended QUEue) - це структура даних, котра забезпечує доступ як зі свого кінця, так і зі свого початку. 
Якщо ви додаєте елементи в один кінець деки, а забираєте з іншого, вона забезпечує принцип FIFO. Якщо додаєте та виймаєте з одного кінця, це буде LIFO.
Імплементація деки є вбудованою в Python.


In [35]:
from collections import deque

my_deque = deque()

In [39]:
for i in range(10, 20):
    my_deque.append(i)

In [37]:
my_deque.append(1)
my_deque.append(2)
my_deque.append(3)

In [34]:
my_deque.popleft()

15

In [40]:
my_deque.extendleft(range(10))

In [45]:
my_deque.pop()

15

In [59]:
my_deque.popleft()

6

Важливою властивістю трьох розглянутих вище структур даних є те, що __всі операції з ними__ мають асимптотичну оцінку по часу О(1)